In [9]:
from sec_processing.utils import *

def main(ticker: str, report_type: str) -> pd.DataFrame:
    """Given a ticker and valid report type, this function returns the report for the latest period"""
    ticker = ticker.upper()
    report_type = report_type.lower()

    valid_report_types = ["balance_sheet", "income_statement", "cash_flow_statement"]
    if report_type not in valid_report_types:
        raise ValueError(f"{report_type} is not a valid report type")

    try:
        acc = get_filtered_filings(ticker, form_type="10-Q", just_accession_numbers=True)
        acc_num = acc.iloc[0].replace('-', '')
    except IndexError:
        raise ValueError(f"There was a problem getting filings for {ticker}")

    statement = process_one_statement(ticker, acc_num, report_type)
    label_dict = get_label_dictionary(ticker, headers)
    statement_df = rename_statement(statement, label_dict)
    print(statement_df)
    return statement_df

In [27]:
ticker = "GOOG"
report_type = "income_statement"
df = main(ticker, report_type)

# Extract EBT and Income Tax rows
ebt = df.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest']
tax_expense = df.loc['Income Tax Expense (Benefit)']

# Calculate Effective Tax Rate
tc = tax_expense / ebt

# Optional: handle negative EBT to avoid nonsensical negative tax rates
tc_clean = tc.apply(lambda x: max(0, x))

print("Effective Tax Rate (Tc) per report date:")
print(tc_clean)


Effective Tax Rate (Tc) per report date:
2025-03-31    0.173467
2024-03-31    0.164330
dtype: float64
